# Chapter 07-01: Attention 메카니즘

## 학습 목표
- Attention 메카니즘의 핵심 수식과 직관적 의미를 이해한다
- NumPy로 Scaled Dot-Product Attention을 직접 구현한다
- TensorFlow/Keras의 Attention 및 MultiHeadAttention 레이어를 사용한다
- Self-Attention과 Cross-Attention의 차이를 구별한다

## 목차
1. Attention 수식 이해
2. NumPy 수동 구현
3. Attention Weight 시각화
4. `tf.keras.layers.Attention` 사용법
5. `tf.keras.layers.MultiHeadAttention` 사용법
6. 정리

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib

# 한글 폰트 설정 (macOS)
matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

print(f'TensorFlow 버전: {tf.__version__}')
print(f'NumPy 버전: {np.__version__}')

## 1. Attention 수식 이해

### Scaled Dot-Product Attention

$$\text{Attention}(Q,K,V) = \text{softmax}\!\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

### 각 행렬의 역할

| 기호 | 이름 | 역할 |
|------|------|------|
| $Q$ | Query  | 현재 위치가 다른 위치들에게 "무엇을 원하는가" 질문 |
| $K$ | Key    | 각 위치가 가진 "내용 설명" — Query와 비교되는 레이블 |
| $V$ | Value  | 실제로 가져올 정보 — Attention Weight에 따라 가중합 |

### $\sqrt{d_k}$로 나누는 이유

$d_k$ (Key 차원)가 커질수록 내적 $QK^T$의 분산이 $d_k$배 증가한다.  
값이 극단적으로 커지면 softmax 출력이 0 또는 1에 집중되어 **기울기 소실**이 발생한다.  
$\sqrt{d_k}$로 나누어 분산을 1로 안정화한다.

### 직관적 비유

> 도서관(V)에서 원하는 책을 찾는 과정:  
> 내 검색어(Q)와 각 책의 색인 키워드(K)를 비교해 유사도를 계산하고,  
> 유사도(Attention Weight)에 따라 책의 내용(V)을 가중합하여 가져온다.

## 2. NumPy로 Attention 수동 구현

In [ ]:
def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Scaled Dot-Product Attention 수동 구현
    
    Args:
        Q: Query 행렬  shape = (..., seq_q, d_k)
        K: Key 행렬    shape = (..., seq_k, d_k)
        V: Value 행렬  shape = (..., seq_k, d_v)
        mask: 선택적 마스크 (패딩 또는 미래 위치 차단)
    Returns:
        output: 가중합 결과  shape = (..., seq_q, d_v)
        weights: Attention 가중치  shape = (..., seq_q, seq_k)
    """
    d_k = Q.shape[-1]  # Key 차원
    
    # Step 1: Q와 K의 내적 계산 → 유사도 점수
    scores = np.matmul(Q, K.transpose(-2, -1))  # (..., seq_q, seq_k)
    
    # Step 2: 스케일링 — sqrt(d_k)로 나누어 기울기 안정화
    scores = scores / np.sqrt(d_k)
    
    # Step 3: 마스크 적용 (선택)
    if mask is not None:
        scores = scores + (mask * -1e9)  # 마스킹 위치를 매우 작은 값으로
    
    # Step 4: Softmax → 확률 분포(Attention Weight)
    # 수치 안정성을 위해 max를 빼고 softmax 계산
    scores_max = np.max(scores, axis=-1, keepdims=True)
    exp_scores = np.exp(scores - scores_max)
    weights = exp_scores / np.sum(exp_scores, axis=-1, keepdims=True)
    
    # Step 5: Value와 가중합
    output = np.matmul(weights, V)  # (..., seq_q, d_v)
    
    return output, weights


# 예시: 시퀀스 길이=5, d_k=d_v=8
np.random.seed(42)
seq_len = 5
d_k = 8
d_v = 8

Q = np.random.randn(seq_len, d_k)  # (5, 8)
K = np.random.randn(seq_len, d_k)  # (5, 8)
V = np.random.randn(seq_len, d_v)  # (5, 8)

output, weights = scaled_dot_product_attention(Q, K, V)

print('Q shape:', Q.shape)
print('K shape:', K.shape)
print('V shape:', V.shape)
print('출력 shape:', output.shape)
print('Attention Weight shape:', weights.shape)
print()
print('Attention Weights (각 행의 합 = 1):')
print(np.round(weights, 3))
print('각 행의 합:', np.round(weights.sum(axis=-1), 6))

## 3. Attention Weight 히트맵 시각화

In [ ]:
# 문장 예시 토큰 (시각화용 레이블)
tokens = ['나는', '오늘', '학교에', '갔다', '.']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# --- 왼쪽: 현재 Attention Weight ---
ax = axes[0]
im = ax.imshow(weights, cmap='Blues', vmin=0, vmax=1)
ax.set_xticks(range(seq_len))
ax.set_yticks(range(seq_len))
ax.set_xticklabels(tokens, fontsize=11)
ax.set_yticklabels(tokens, fontsize=11)
ax.set_xlabel('Key (어떤 위치를 참조하는가)', fontsize=10)
ax.set_ylabel('Query (어느 위치의 Attention인가)', fontsize=10)
ax.set_title('Scaled Dot-Product Attention Weight', fontsize=12)
plt.colorbar(im, ax=ax)

# 각 셀에 값 표시
for i in range(seq_len):
    for j in range(seq_len):
        ax.text(j, i, f'{weights[i, j]:.2f}',
                ha='center', va='center', fontsize=8,
                color='white' if weights[i, j] > 0.5 else 'black')

# --- 오른쪽: 임의 마스크 적용 (미래 위치 차단 = Causal Mask) ---
causal_mask = np.triu(np.ones((seq_len, seq_len)), k=1)  # 상삼각 행렬
_, masked_weights = scaled_dot_product_attention(Q, K, V, mask=causal_mask)

ax2 = axes[1]
im2 = ax2.imshow(masked_weights, cmap='Oranges', vmin=0, vmax=1)
ax2.set_xticks(range(seq_len))
ax2.set_yticks(range(seq_len))
ax2.set_xticklabels(tokens, fontsize=11)
ax2.set_yticklabels(tokens, fontsize=11)
ax2.set_xlabel('Key', fontsize=10)
ax2.set_ylabel('Query', fontsize=10)
ax2.set_title('Causal (마스크 적용) Attention Weight', fontsize=12)
plt.colorbar(im2, ax=ax2)

for i in range(seq_len):
    for j in range(seq_len):
        ax2.text(j, i, f'{masked_weights[i, j]:.2f}',
                 ha='center', va='center', fontsize=8,
                 color='white' if masked_weights[i, j] > 0.5 else 'black')

plt.suptitle('Attention Weight 히트맵 비교', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print('Causal Mask (상삼각 = 미래 위치):')
print(causal_mask)

## 4. `tf.keras.layers.Attention` 레이어 사용법

`tf.keras.layers.Attention`은 Bahdanau-style(additive) 또는 dot-product Attention을 제공한다.  
주로 Seq2Seq 디코더의 Cross-Attention에 사용된다.

In [ ]:
# tf.keras.layers.Attention 기본 사용법
batch_size = 2
seq_q = 4   # Query 시퀀스 길이 (디코더)
seq_k = 6   # Key/Value 시퀀스 길이 (인코더)
dim = 16    # 임베딩 차원

# 랜덤 입력 텐서 생성
query_input = tf.random.normal((batch_size, seq_q, dim))
key_input   = tf.random.normal((batch_size, seq_k, dim))
value_input = tf.random.normal((batch_size, seq_k, dim))

# Attention 레이어 생성 (use_scale=True → Scaled Dot-Product)
attention_layer = tf.keras.layers.Attention(use_scale=True)

# 호출: [query, value] 또는 [query, value, key] 순서로 전달
attention_output = attention_layer([query_input, value_input, key_input])

print('입력 Query shape:', query_input.shape)   # (2, 4, 16)
print('입력 Key shape:  ', key_input.shape)     # (2, 6, 16)
print('입력 Value shape:', value_input.shape)   # (2, 6, 16)
print('Attention 출력 shape:', attention_output.shape)  # (2, 4, 16)

# Attention Score(Weight)도 반환받기
attention_output2, attention_scores = attention_layer(
    [query_input, value_input, key_input],
    return_attention_scores=True
)
print('Attention Scores shape:', attention_scores.shape)  # (2, 4, 6)

# 모델에서 사용하는 예시
print('\n--- Functional API 예시 ---')
query_in = tf.keras.Input(shape=(seq_q, dim), name='query')
key_in   = tf.keras.Input(shape=(seq_k, dim), name='key')
val_in   = tf.keras.Input(shape=(seq_k, dim), name='value')
attn_out = tf.keras.layers.Attention(use_scale=True)([query_in, val_in, key_in])
model = tf.keras.Model(inputs=[query_in, key_in, val_in], outputs=attn_out)
model.summary()

## 5. `tf.keras.layers.MultiHeadAttention` 레이어 사용법

Multi-Head Attention은 여러 개의 Attention을 병렬로 수행하여  
서로 다른 표현 부분 공간(representation subspace)에서 정보를 동시에 학습한다.

$$\text{MultiHead}(Q,K,V) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h)W^O$$
$$\text{head}_i = \text{Attention}(QW_i^Q,\; KW_i^K,\; VW_i^V)$$

In [ ]:
# MultiHeadAttention 기본 사용법
num_heads = 4   # Attention 헤드 수
key_dim   = 8   # 각 헤드의 Key 차원 (d_k)
seq_len   = 10  # 시퀀스 길이
embed_dim = 32  # 임베딩 차원 (= num_heads × key_dim)

# 레이어 생성
mha_layer = tf.keras.layers.MultiHeadAttention(
    num_heads=num_heads,
    key_dim=key_dim,
    value_dim=key_dim,   # Value 차원 (생략하면 key_dim과 동일)
    dropout=0.0,
    name='multi_head_attention'
)

# 입력 텐서
x = tf.random.normal((batch_size, seq_len, embed_dim))

# Self-Attention: Query = Key = Value = x
self_attn_output, self_attn_weights = mha_layer(
    query=x,
    key=x,
    value=x,
    return_attention_scores=True
)
print('Self-Attention 출력 shape:', self_attn_output.shape)  # (2, 10, 32)
print('Self-Attention Weight shape:', self_attn_weights.shape)  # (2, 4, 10, 10)
print('  → (batch, num_heads, seq_q, seq_k)')

# Cross-Attention: Query는 디코더, Key/Value는 인코더
encoder_output = tf.random.normal((batch_size, 6, embed_dim))  # 인코더 출력
decoder_query  = tf.random.normal((batch_size, 4, embed_dim))  # 디코더 상태

cross_attn_output = mha_layer(
    query=decoder_query,
    key=encoder_output,
    value=encoder_output
)
print('\nCross-Attention 출력 shape:', cross_attn_output.shape)  # (2, 4, 32)

# 각 헤드의 Attention Weight 시각화 (첫 번째 샘플)
fig, axes = plt.subplots(1, num_heads, figsize=(16, 3))
for i in range(num_heads):
    ax = axes[i]
    w = self_attn_weights[0, i].numpy()  # (seq_len, seq_len)
    im = ax.imshow(w, cmap='viridis', vmin=0, vmax=1)
    ax.set_title(f'Head {i+1}', fontsize=10)
    ax.set_xlabel('Key', fontsize=8)
    if i == 0:
        ax.set_ylabel('Query', fontsize=8)

fig.suptitle('Multi-Head Self-Attention Weights (헤드별 비교)', fontsize=12)
plt.colorbar(im, ax=axes[-1])
plt.tight_layout()
plt.show()

## 6. 정리

### Self-Attention vs Cross-Attention 비교

| 구분 | Self-Attention | Cross-Attention |
|------|---------------|----------------|
| Q, K, V 출처 | 동일한 시퀀스 | Q: 디코더, K/V: 인코더 |
| 사용 위치 | Transformer Encoder, GPT | Transformer Decoder, Seq2Seq |
| 목적 | 시퀀스 내 각 토큰이 서로를 참조 | 디코더가 인코더 정보를 참조 |
| 예시 | 문장 내 단어 간 관계 파악 | 번역 시 소스-타겟 정렬 |

### 핵심 요약

- **Attention**은 시퀀스 내 임의 위치 간 직접 연결을 가능하게 한다 (RNN의 장거리 의존성 문제 해결)
- **Scaling** ($\div\sqrt{d_k}$)은 내적 값의 분산 폭발을 방지한다
- **Multi-Head**는 여러 관점에서 동시에 Attention을 계산하여 표현력을 높인다
- **Causal Mask**는 Decoder에서 미래 정보 누출을 방지한다

### 다음 챕터 예고

**Chapter 07-02: Transformer Basics** — Positional Encoding, Encoder Block, Transformer를 직접 구현하고 텍스트 분류에 적용한다.